# Init

In [ ]:
# imports
import gurobipy as gp
from gurobipy import GRB
from collections import defaultdict
import numpy as np
from collections import deque
import random

In [ ]:
OPTIMAL_SOLUTION = 0 # used to evaluate gap between math model and heuristics

---
#### Load data

In [ ]:
num_video = 0
num_endpoint = 0
num_req_descriptions = 0
num_server = 0

cache_capacity = 0
video_size = []

latency = defaultdict(lambda: defaultdict(int))     # [endpoint][cache/datacenter] = latency
reqs = defaultdict(lambda: defaultdict(int))        # [endpoint][video] = num reqs

# dataset = "dataset/videos_worth_spreading.in"
dataset = "dataset/me_at_the_zoo.in"
# dataset = "dataset/custom.in"
# dataset = "dataset/minimal.in"


In [ ]:
status = 0
curr_endpoint_index = 0
num_connected_cache = 0
with open(dataset, "r") as f:
    for line_content in f:
        line = line_content.split()

        if status ==0:                                  # get counters
            num_video = int(line[0])
            num_endpoint = int(line[1])
            num_req_descriptions = int(line[2])
            num_server = int(line[3])
            cache_capacity = int(line[4])
            status = 1

        elif status == 1:                               # get video dims
            for size in line:
                video_size.append(int(size))
            status = 2

        elif status == 2:                               # get datacenter latency and connected cache number
            data_center_latency = int(line[0])
            latency[curr_endpoint_index][num_server] = data_center_latency
            
            num_connected_cache = int(line[1])
            if not num_connected_cache:
                curr_endpoint_index = curr_endpoint_index + 1
                if curr_endpoint_index == num_endpoint:
                    status = 4
            else:
                status = 3
        
        elif status == 3:                                  # get cache latency
            cache_index = int(line[0])
            cache_latency = int(line[1])
            latency[curr_endpoint_index][cache_index] = cache_latency
            
            num_connected_cache = num_connected_cache - 1
            if not num_connected_cache:
                curr_endpoint_index = curr_endpoint_index + 1
                if curr_endpoint_index == num_endpoint:
                    status = 4
                else:
                    status = 2
        
        elif status == 4:                                   # take num requests
            video_index = int(line[0])
            curr_endpoint_index = int(line[1])
            num_reqs = int(line[2])
            reqs[curr_endpoint_index][video_index] = num_reqs                         

In [ ]:
# Common indexes
endpoint_index = range(num_endpoint)
server_index = range(num_server + 1) # I've modelled datacenter as last server
video_index = range(num_video)

In [ ]:
print(f"num video: {num_video}, num endpoints {num_endpoint}, req descriptions {num_req_descriptions}, num cache {num_server}, dim {cache_capacity}")
print(f"video sized: {video_size}")
print(f"latencies: {latency}")
print(f"reqs: {reqs}")

---
# Math model for Guroby

In [ ]:
model = gp.Model("YoutubeCache")

# DECISION VARS
x = model.addVars(endpoint_index, server_index, video_index, vtype=gp.GRB.BINARY, name="x")
y = model.addVars(server_index, video_index, vtype=gp.GRB.BINARY, name="y")

# OBJECTIVE FUNCTION
obj = gp.quicksum(latency[e][s]*reqs[e][v]*x[e,s,v] for e in endpoint_index for s in server_index for v in video_index)
# the + y[s,v] it's used just to not let place useless video in cache server (but is not needed for this problem)
# obj = gp.quicksum((latency[e][s]*reqs[e][v]*x[e,s,v] + y[s,v])for e in endpoint_index for s in server_index for v in video_index)
model.setObjective(obj, GRB.MINIMIZE)


# CONSTRAINTS
constr = (gp.quicksum(x[e,s,v] for e in endpoint_index)  <= num_endpoint*y[s,v] for s in server_index for v in video_index )
model.addConstrs(constr, name="if video v available on server s")

constr = ( gp.quicksum( x[e,s,v] for s in server_index ) == (1 if reqs[e][v] else 0) for e in endpoint_index for v in video_index ) # datacenter excluded 
model.addConstrs(constr, name="every request must be satisfied")

constr = ( gp.quicksum(video_size[v] * y[s,v] for v in video_index) <= cache_capacity for s in server_index[:-1] ) # -1 because datacenter have all the video
model.addConstrs(constr, name="cache capacity")


constr = ( y[num_server,v] == 1 for v in video_index ) # cache servers are from 0 to s-1, s index (num_server) is for datacenter
model.addConstrs(constr, name="Datacenter have all videos")

constr = ( gp.quicksum( x[e,s,v] for v in video_index ) <= (num_video*latency[e][s]) for e in endpoint_index for s in server_index[:-1] ) # -1 because datacenter have all the video
model.addConstrs(constr, name="video v must be available on server s to be selected")


In [ ]:
# Optimize the model
model.optimize()

# model.display()

In [ ]:
# Print model output

def print_full_output():
    print("Optimal X [endpoint, server, video] values:")
    for e in endpoint_index:
        for s in server_index:
            for v in video_index:
                print(f"X[{e},{s},{v}] * {latency[e][s]} = {x[e,s,v]}")
    print("\nOptimal Y [server, video] values:")
    for s in server_index:
        for v in video_index:
            print(f"Y[{s},{v}] = {y[s,v]}")

def print_concise_output():
    print("Optimal X [endpoint, server, video] values:")
    for e in endpoint_index:
        for s in server_index:
            for v in video_index:
                if x[e,s,v].x:
                    print(f"X[{e},{s},{v}] * {latency[e][s]} = {x[e,s,v]}")
    print("\nOptimal Y [server, video] values:")
    for s in server_index:
        for v in video_index:
            if y[s,v].x:
                print(f"Y[{s},{v}] = {y[s,v]}")

In [ ]:
# results
if model.status == gp.GRB.OPTIMAL:
    print("\nOptimization successful!")
    # print_full_output()
    print_concise_output()
    print(f"\nOptimal objective value: {model.objVal}")
    OPTIMAL_SOLUTION = model.ObjVal
elif model.status == gp.GRB.INFEASIBLE:
    print("Model is infeasible.")
elif model.status == gp.GRB.UNBOUNDED:
    print("Model is unbounded.")
else:
    print(f"Optimization ended with status {model.status}")

---
# Heuristics

In [ ]:
## Common
def compute_obj_func(x):
    return sum(latency[e][s]*reqs[e][v]*x[e,s,v] for e in endpoint_index for s in server_index for v in video_index)

x_sol = []
y_sol = []

## Basics

### 1. place video with highest request number in nearest cache when possible and place all videos for the endpoint in the order that we get
order video by request number, and for every endpoint retrieven from this ordered list place all its requested videos in best available caches for it

In [ ]:
E_IND = 0
V_IND = 1

# Sort v indexes by descending value for endpoint e
sorted_reqs = []
for e in range(len(reqs)):
    sorted_vs = sorted(
        [v for v in reqs[e] if reqs[e][v] != 0],
        key=lambda v: reqs[e][v],
        reverse=True
    )
    sorted_reqs.extend((e, v) for v in sorted_vs)

# Sort server s latency for endpoint e
sorted_latency = defaultdict(list)
for e in latency:
    sorted_s = sorted(
        [s for s in latency[e] if latency[e][s] != 0],
        key=lambda s: latency[e][s]
    )
    sorted_latency[e] = sorted_s


# use a list to keep current cache capacity (will be decreased every time a video is placed in cache)
curr_capacity = [cache_capacity for _ in range(num_server)]
curr_capacity.append(float('inf')) # datacenter doesn't have capacity

# create vars (simil guroby, used numpy for efficiency)
x = np.zeros((num_endpoint, (num_server+1), num_video)) 
y = np.zeros(((num_server+1), num_video)) 
y[num_server, :] = 1 # datacenter keep all the videos

for req in sorted_reqs:
    req_endpoint = req[E_IND]
    req_video = req[V_IND]
    req_video_size = video_size[req_video]

    for curr_cache_index in sorted_latency[req_endpoint]:
        if y[curr_cache_index, req_video]:
            x[req_endpoint, curr_cache_index, req_video] = 1
            break
        else:
            if curr_capacity[curr_cache_index] > req_video_size:
                curr_capacity[curr_cache_index] -= req_video_size
                y[curr_cache_index, req_video] = 1
                x[req_endpoint, curr_cache_index, req_video] = 1
                break

# print("X")
# print(x)
# print("Y")
# print(y)
APPROX_RESULT = compute_obj_func(x)
GAP = ( abs(OPTIMAL_SOLUTION - APPROX_RESULT) / OPTIMAL_SOLUTION ) * 100
print(f"APPROX RESULT: {APPROX_RESULT} - GAP: {GAP}% - OPTIMAL RESULT {OPTIMAL_SOLUTION}")
x_sol.append((x, APPROX_RESULT))
y_sol.append((y, APPROX_RESULT))


### 2. place video with highest request number in nearest cache + round robin (every iteration change endpoint)
order video by request number, use a round robin schedulo to choose an endpoint retrieven from this ordered list and place its remaining requested video in best available cache

In [ ]:
E_IND = 0
V_IND = 1

# Sort v indexes by descending value for endpoint e
sorted_reqs = defaultdict(list)
for e in range(len(reqs)):
    sorted_vs = sorted(
        [v for v in reqs[e] if reqs[e][v] != 0],
        key=lambda v: reqs[e][v],
        reverse=True
    )
    sorted_reqs[e] = sorted_vs

# Sort server s latency for endpoint e
sorted_latency = defaultdict(list)
for e in latency:
    sorted_s = sorted(
        [s for s in latency[e] if latency[e][s] != 0],
        key=lambda s: latency[e][s]
    )
    sorted_latency[e] = sorted_s


# use a list to keep current cache capacity (will be decreased every time a video is placed in cache)
curr_capacity = [cache_capacity for _ in range(num_server)]
curr_capacity.append(float('inf')) # datacenter doesn't have capacity

# create vars (simil guroby, used numpy for efficiency)
x = np.zeros((num_endpoint, (num_server+1), num_video))
y = np.zeros(((num_server+1), num_video))
y[num_server, :] = 1 # datacenter keep all the videos

Done = False
endpoints_req_index = [0 for _ in server_index]
while not Done:
    Done = True
    for curr_endpoint in endpoint_index:
        curr_endpoint_req_index = endpoints_req_index[curr_endpoint]
        
        if curr_endpoint_req_index < len(sorted_reqs[curr_endpoint]):
            Done = False # There could still be reqs not satisfied other than this
            req_video = sorted_reqs[curr_endpoint][curr_endpoint_req_index]
            req_video_size = video_size[req_video]

            for curr_cache_index in sorted_latency[curr_endpoint]:
                if y[curr_cache_index, req_video]:
                    x[curr_endpoint, curr_cache_index, req_video] = 1
                    break
                else:
                    if curr_capacity[curr_cache_index] > req_video_size:
                        curr_capacity[curr_cache_index] -= req_video_size
                        y[curr_cache_index, req_video] = 1
                        x[curr_endpoint, curr_cache_index, req_video] = 1
                        break
                    
        endpoints_req_index[curr_endpoint] += 1

# print("X")
# print(x)
# print("Y")
# print(y)
APPROX_RESULT = compute_obj_func(x)
GAP = ( abs(OPTIMAL_SOLUTION - APPROX_RESULT) / OPTIMAL_SOLUTION ) * 100
print(f"APPROX RESULT: {APPROX_RESULT} - GAP: {GAP}% - OPTIMAL RESULT {OPTIMAL_SOLUTION}")
x_sol.append((x, APPROX_RESULT))
y_sol.append((y, APPROX_RESULT))


### 3. place video with highest request number in nearest cache when possible
(only order by request number without considering the endpoints, pratically place cache video in the best server order by request number withouth reasoning on endpoints)

In [ ]:
E_IND = 0
V_IND = 1

# Sort v indexes by descending value for endpoint e
sorted_reqs = []
sorted_reqs = sorted(
    [(e, v) for e in range(len(reqs)) for v in reqs[e] if reqs[e][v] != 0],
    key=lambda pair: reqs[pair[0]][pair[1]],
    reverse=True
)

# Sort server s latency for endpoint e
sorted_latency = defaultdict(list)
for e in latency:
    sorted_s = sorted(
        [s for s in latency[e] if latency[e][s] != 0],
        key=lambda s: latency[e][s]
    )
    sorted_latency[e] = sorted_s


# use a list to keep current cache capacity (will be decreased every time a video is placed in cache)
curr_capacity = [cache_capacity for _ in range(num_server)]
curr_capacity.append(float('inf')) # datacenter doesn't have capacity

# create vars (simil guroby, used numpy for efficiency)
x = np.zeros((num_endpoint, (num_server+1), num_video))
y = np.zeros(((num_server+1), num_video))
y[num_server, :] = 1 # datacenter keep all the videos

for req_endpoint,req_video in sorted_reqs:
    req_video_size = video_size[req_video]

    for curr_cache_index in sorted_latency[req_endpoint]:
        if y[curr_cache_index, req_video]:
            x[req_endpoint, curr_cache_index, req_video] = 1
            break
        else:
            if curr_capacity[curr_cache_index] > req_video_size:
                curr_capacity[curr_cache_index] -= req_video_size
                y[curr_cache_index, req_video] = 1
                x[req_endpoint, curr_cache_index, req_video] = 1
                break

# print("X")
# print(x)
# print("Y")
# print(y)
APPROX_RESULT = compute_obj_func(x)
GAP = ( abs(OPTIMAL_SOLUTION - APPROX_RESULT) / OPTIMAL_SOLUTION ) * 100
print(f"APPROX RESULT: {APPROX_RESULT} - GAP: {GAP}% - OPTIMAL RESULT {OPTIMAL_SOLUTION}")
x_sol.append((x, APPROX_RESULT))
y_sol.append((y, APPROX_RESULT))


### 4. Place video ordered by popularity in cache with most connected endpoints for that video
Order video by request number and place ordered video in the cache connected with most endpoints that have requested that specific video

In [ ]:
video_req_count = [0 for _ in video_index]
latency_sum = defaultdict(lambda: defaultdict(lambda: defaultdict(int))) # used to find best cache to place a video [server][video][latency sum / num endpoint / score]
LATENCY_INDEX = 0
NUM_ENDPOINT_CONNECTED_INDEX = 1
SCORE_INDEX = 2

for curr_endpoint_index, endpoint_reqs in reqs.items():
    for curr_video_index, req_num in endpoint_reqs.items():
        if req_num: # check if requests from endpoint for the video exists
            video_req_count[curr_video_index] += req_num
            for curr_server_index, lat in latency[curr_endpoint_index].items():
                if curr_server_index != num_server and lat:
                    latency_sum[curr_server_index][curr_video_index][LATENCY_INDEX] += lat
                    latency_sum[curr_server_index][curr_video_index][NUM_ENDPOINT_CONNECTED_INDEX] += 1
                    latency_sum[curr_server_index][curr_video_index][SCORE_INDEX] = (
                        latency_sum[curr_server_index][curr_video_index][NUM_ENDPOINT_CONNECTED_INDEX] 
                        /
                        latency_sum[curr_server_index][curr_video_index][LATENCY_INDEX]
                    )
            
sorted_video_indexes = sorted(range(num_video), key=lambda i: video_req_count[i], reverse=True)

# use a list to keep current cache capacity (will be decreased every time a video is placed in cache)
curr_capacity = [cache_capacity for _ in range(num_server)]
curr_capacity.append(float('inf')) # datacenter doesn't have capacity
# print(curr_capacity)

# create vars (simil guroby, used numpy for efficiency)
x = np.zeros((num_endpoint, (num_server+1), num_video)) # e take v from s
y = np.zeros(((num_server+1), num_video)) # v is in s
y[num_server, :] = 1 # datacenter keep all the videos

# Sort server s latency for endpoint e
sorted_latency = defaultdict(list)
for e in latency:
    # Sort v indices by ascending value for this e
    sorted_s = sorted(
        [s for s in latency[e] if latency[e][s] != 0],
        key=lambda s: latency[e][s]
    )
    sorted_latency[e] = sorted_s


for curr_video_index in sorted_video_indexes:
    # now sort caches to get the ones with most connected endpoints that have requested video curr_video_index
    for curr_server_index in sorted(
        [
            i
            for i in range(len(latency_sum))
            if latency_sum[i][curr_video_index][NUM_ENDPOINT_CONNECTED_INDEX] != 0
        ],
        key=lambda i: latency_sum[i][curr_video_index][NUM_ENDPOINT_CONNECTED_INDEX],
        reverse=True
    ):
        if not y[curr_server_index, curr_video_index]:
            if curr_capacity[curr_server_index] > video_size[curr_video_index]:
                curr_capacity[curr_server_index] -= video_size[curr_video_index]
                y[curr_server_index, curr_video_index] = 1
                break

# iterate trough all reqs to check to what server the endpoint should request the video
for req_endpoint,req_videos in reqs.items():    
    for curr_video_index in req_videos:
        if reqs[req_endpoint][curr_video_index]:
            for curr_server_index in sorted_latency[req_endpoint]:
                if y[curr_server_index, curr_video_index]:
                    x[req_endpoint, curr_server_index, curr_video_index] = 1
                    break

# print("X")
# print(x)
# print("Y")
# print(y)
APPROX_RESULT = compute_obj_func(x)
GAP = ( abs(OPTIMAL_SOLUTION - APPROX_RESULT) / OPTIMAL_SOLUTION ) * 100
print(f"APPROX RESULT: {APPROX_RESULT} - GAP: {GAP}% - OPTIMAL RESULT {OPTIMAL_SOLUTION}")
x_sol.append((x, APPROX_RESULT))
y_sol.append((y, APPROX_RESULT))


### 5. place video based on connected endpoint most requested videos
for every caches, place their most requested video

In [ ]:
E_IND = 0
V_IND = 1

# Sort v indexes by descending value for endpoint e
sorted_reqs = []
sorted_reqs = sorted(
    [(e, v) for e in range(len(reqs)) for v in reqs[e] if reqs[e][v] != 0],
    key=lambda pair: reqs[pair[0]][pair[1]],
    reverse=True
)

# Sort server s latency for endpoint e
sorted_latency = defaultdict(list)
for e in latency:
    sorted_s = sorted(
        [s for s in latency[e] if latency[e][s] != 0],
        key=lambda s: latency[e][s]
    )
    sorted_latency[e] = sorted_s


# use a list to keep current cache capacity (will be decreased every time a video is placed in cache)
curr_capacity = [cache_capacity for _ in range(num_server)]
curr_capacity.append(float('inf')) # datacenter doesn't have capacity

# create vars (simil guroby, used numpy for efficiency)
x = np.zeros((num_endpoint, (num_server+1), num_video)) # e take v from s
y = np.zeros(((num_server+1), num_video)) # v is in s
y[num_server, :] = 1 # datacenter keep all the videos


# get total possible reqs for any cache server from its connected endpoints
server_total_reqs = defaultdict(lambda: defaultdict(int))
for req_endpoint,req_videos in reqs.items():
    for curr_server_index, lat in latency[req_endpoint].items():
                if curr_server_index != num_server and lat:
                    for curr_video_index in req_videos.keys():
                        server_total_reqs[curr_server_index][curr_video_index] += req_videos[curr_video_index]


# place video in caches based on request counts
for curr_server_index in server_index[:-1]:
    sorted_indexes_only = [
        video_index
        for video_index, count in sorted(
            server_total_reqs[curr_server_index].items(),
            key=lambda x: x[1],
            reverse=True
        )
        if count > 0
    ]
    for curr_video_index in sorted_indexes_only:
        if not y[curr_server_index, curr_video_index] and curr_capacity[curr_server_index] > video_size[curr_video_index]:
            curr_capacity[curr_server_index] -= video_size[curr_video_index]
            y[curr_server_index, curr_video_index] = 1


# iterate trough all reqs to check to what server the endpoint should request the video
for req_endpoint,req_videos in reqs.items():    
    for curr_video_index in req_videos:
        if reqs[req_endpoint][curr_video_index]:
            for curr_server_index in sorted_latency[req_endpoint]:
                if y[curr_server_index, curr_video_index]:
                    x[req_endpoint, curr_server_index, curr_video_index] = 1
                    break

# print("X")
# print(x)
# print("Y")
# print(y)
APPROX_RESULT = compute_obj_func(x)
GAP = ( abs(OPTIMAL_SOLUTION - APPROX_RESULT) / OPTIMAL_SOLUTION ) * 100
print(f"APPROX RESULT: {APPROX_RESULT} - GAP: {GAP}% - OPTIMAL RESULT {OPTIMAL_SOLUTION}")
x_sol.append((x, APPROX_RESULT))
y_sol.append((y, APPROX_RESULT))

## Local Search

### tabu search

In [ ]:
def get_best_x(y):
    x = np.zeros((num_endpoint, (num_server+1), num_video))

    # Sort server s latency for endpoint e
    sorted_latency = defaultdict(list)
    for e in latency:
        sorted_s = sorted(
            [s for s in latency[e] if latency[e][s] != 0],
            key=lambda s: latency[e][s]
        )
        sorted_latency[e] = sorted_s
    
    for curr_endpoint_index in endpoint_index:
        for curr_video_index in reqs[curr_endpoint_index]:
            best_server_index = num_server  # default to datacenter
            
            for curr_server_index in sorted_latency[curr_endpoint_index]:                    
                if y[curr_server_index][curr_video_index]:
                    best_server_index = curr_server_index
                    break

            x[curr_endpoint_index][best_server_index][curr_video_index] = 1

    return x

In [ ]:
def tabu_search_toggle(x, y, num_iters=1000, tabu_list_dim=10):
    tabu_list = deque(maxlen=tabu_list_dim)
    intensification_list = []
    
    # Start with initial solution
    best_x = np.copy(x)
    best_y = np.copy(y)
    best_obj_val = compute_obj_func(best_x)

    for iteration in range(num_iters):
        neighborhood = []

        # Generate neighbor solutions
        for curr_server_index in server_index[:-1]:  # Only cache servers
            for curr_video_index in video_index:
                move = (curr_server_index, curr_video_index)
                if move in tabu_list:
                    continue

                # Try toggling video v in cache s
                new_y = np.copy(y)
                new_y[curr_server_index][curr_video_index] = 1 - new_y[curr_server_index][curr_video_index]

                # Check cache capacity constraint
                curr_video_size = sum(video_size[v] for v in video_index if new_y[curr_server_index][v])
                if curr_video_size > cache_capacity:
                    continue

                # Generate new x according to new y
                new_x = get_best_x(new_y)

                obj_val = compute_obj_func(new_x)

                neighborhood.append((obj_val, new_x, new_y, move))

        if not neighborhood:
            break

        # Choose best neighbor
        neighborhood.sort(key=lambda tup: tup[0])  # Sort by delay
        obj_val, new_x, new_y, move = neighborhood[0]

        if obj_val < best_obj_val:
            best_obj_val = obj_val
            best_x = new_x
            best_y = new_y
            intensification_list.append((best_obj_val, best_x, best_y))
            intensification_list = sorted(intensification_list)[:10]  # keep top 10

        # Update current solution
        x = new_x
        y = new_y

        # Update tabu list
        tabu_list.append(move)
        
        # Random intensification
        if iteration % 70 == 0 and intensification_list:
            _, best_x, best_y = random.choice(intensification_list)
            x, y = best_x.copy(), best_y.copy()

    return best_x, best_y, best_obj_val

In [ ]:
def tabu_search_add(x, y, num_iters=1000, tabu_list_dim=10):
    tabu_list = deque(maxlen=tabu_list_dim)
    intensification_list = []
    
    # Start with initial solution
    best_x = np.copy(x)
    best_y = np.copy(y)
    best_obj_val = compute_obj_func(best_x)

    for iteration in range(num_iters):
        neighborhood = []

        # Generate neighbor solutions
        for curr_server_index in server_index[:-1]:  # Only cache servers
            for curr_video_index in video_index:
                move = (curr_server_index, curr_video_index)
                if move in tabu_list:
                    continue

                # Try toggling video v in cache s
                new_y = np.copy(y)
                
                # Try adding current video in current server if not already present
                if new_y[curr_server_index][curr_video_index]:
                    continue
                new_y[curr_server_index][curr_video_index] = 1 

                # Check cache capacity constraint
                curr_video_size = sum(video_size[v] for v in video_index if new_y[curr_server_index][v])
                if curr_video_size > cache_capacity:
                    continue

                # Generate new x according to new y
                new_x = get_best_x(new_y)

                obj_val = compute_obj_func(new_x)

                neighborhood.append((obj_val, new_x, new_y, move))

        if not neighborhood:
            break

        # Choose best neighbor
        neighborhood.sort(key=lambda tup: tup[0])  # Sort by delay
        obj_val, new_x, new_y, move = neighborhood[0]

        if obj_val < best_obj_val:
            best_obj_val = obj_val
            best_x = new_x
            best_y = new_y
            intensification_list.append((best_obj_val, best_x, best_y))
            intensification_list = sorted(intensification_list)[:10]  # keep top 10

        # Update current solution
        x = new_x
        y = new_y

        # Update tabu list
        tabu_list.append(move)
        
        # Random intensification
        if iteration % 70 == 0 and intensification_list:
            _, best_x, best_y = random.choice(intensification_list)
            x, y = best_x.copy(), best_y.copy()

    return best_x, best_y, best_obj_val

In [ ]:
print(f"best sol: {OPTIMAL_SOLUTION}")
tabu_list_dim = np.floor(np.sqrt(num_req_descriptions))

for sol_index in range(len(x_sol)):
    x_tabu_toggle, y_tabu_toggle, obj_tabu_toggle = tabu_search_toggle(x_sol[sol_index][0], y_sol[sol_index][0], tabu_list_dim=tabu_list_dim)
    print(f"old solution: {compute_obj_func(x_sol[sol_index][0])}, tabu sol: {obj_tabu_toggle}")
    
    x_tabu_add, y_tabu_add, obj_tabu_add = tabu_search_add(x_sol[sol_index][0], y_sol[sol_index][0], tabu_list_dim=tabu_list_dim)
    print(f"old solution: {compute_obj_func(x_sol[sol_index][0])}, tabu sol: {obj_tabu_add}")
    
    if obj_tabu_toggle > obj_tabu_add:
        x_sol.append((x_tabu_add, obj_tabu_add))
        y_sol.append((y_tabu_add, obj_tabu_add))
    else:
        x_sol.append((x_tabu_toggle, obj_tabu_toggle))
        y_sol.append((y_tabu_toggle, obj_tabu_toggle))

## Genetic algorithm

#### fitness: minimize delay ( compute_obj_func(x) )
#### initial population: heuristics solutions (x_sol & y_sol)
#### randomization: montecarlo simulation
#### crossover & mutation: on y (update x conseguently get_best_x(y) )

In [ ]:
X_IND = 0
Y_IND = 1
OBJ_IND = 2

POP_SIZE = 50
NGEN = 100
MUT_RATE = 0.001

def is_feasible(y):
    for curr_server_index in server_index:
        # calculate current size when video is placed on specific cache
        curr_tot_size = sum(video_size[curr_video_index] for curr_video_index in video_index if y[curr_server_index, curr_video_index])
        if curr_tot_size < cache_capacity[curr_server_index]:
            return False
    return True

def montecarlo_roulette_individual_selection(current_population, reverse=False):
    if reverse:
        fitness_list = [ curr_individual[OBJ_IND] for curr_individual in current_population ]
    else:
        # 1 / obj func because less delay will have bigger size
        fitness_list = [ (1/curr_individual[OBJ_IND]) for curr_individual in current_population ]
        
    fitness_tot = sum(fitness_list) - 1 #-1 for randint generation
    
    spin = random.randint(0, fitness_tot)
    
    curr_fit = 0
    for curr_individual_index in range(len(current_population)):
        curr_fit += fitness_list[curr_individual_index]
        if spin < curr_fit:
            return current_population[curr_individual_index]

def crossover(parent1, parent2):
    # we need to clone parent or python will reference to them (and if we do mutation we'll do it also on parents)
    y1 = parent1[1].copy()
    y2 = parent2[1].copy()
    
    childs_y = []
    childs = []
    
    # Monosplit crossover
    split = np.random.randint(1, num_video)
    # take y rows [0 to split] from parent 1 and y rows [split to num_server+1] from parent2 
    childs_y.append( np.vstack([y1[:,:split], y2[:,split:]]) )
    
    # kill unfeasible child (spartan way)
    for child_y in childs_y:
        if not is_feasible(child_y):
            continue
        
        child_x = get_best_x(child_y)
        childs.append(child_x, child_y, compute_obj_func(child_x))
    
    return childs

#TODO RECHECK THIS METHOD AND IMPROVE IT
def mutate(childs):

    childs_mutated = childs.copy()
    
    for child in childs:
        child_unmutated = child.copy()
        for curr_server_index in server_index[:-1]:
            for curr_video_index in video_index:
                if random.random() < MUT_RATE:
                    child[Y_IND][curr_server_index][curr_video_index] ^= 1 # toggle mutation
                    
        if is_feasible(y):
            childs_mutated.remove(child_unmutated)
            childs_mutated.append(child)
            
    return childs_mutated

# Main GA loop
def run_ga():
    # an individual is (x, y, obj value)
    population = [(x_sol[curr_individual_index][0], y_sol[curr_individual_index][0], x_sol[curr_individual_index][1]) for curr_individual_index in range(len(y_sol))]
    
    for gen in range(NGEN):
        for _ in range(3): # generate 6 individual every iteration
            parent_1 = montecarlo_roulette_individual_selection(population)
            parent_2 = montecarlo_roulette_individual_selection(population)
            childs = crossover(parent_1, parent_2)
            childs = mutate(childs)
            population.append(childs)
        
        for _ in range(2): # remove 2 individual every iteration
            dead = montecarlo_roulette_individual_selection(population, reverse=True)
            population.remove(dead)
        
        best = min(population, key=lambda ind: compute_obj_func(ind[0]))
        print(f"Gen {gen}: Best delay = {compute_obj_func(best[0])}")
    return min(population, key=lambda ind: compute_obj_func(ind[0]))
